# 2019 Data Science Bowl

* В этом соревновании необходимо предсказать результаты детей, выполняющих задачи в специальным приложении, по их истории взаимодействия с обучающей частью приложения и ранее решенным тестам.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

from sklearn.model_selection import RandomizedSearchCV
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
from IPython.display import HTML
import warnings
pd.set_option('max_columns', 100)
warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
my_pal = sns.color_palette(n_colors=10)

In [ ]:
train = pd.read_csv('../input/data-science-bowl-2019/train.csv')  # действия детей в приложении
train_labels = pd.read_csv('../input/data-science-bowl-2019/train_labels.csv')  # их результаты в тестах
test = pd.read_csv('../input/data-science-bowl-2019/test.csv')  
specs = pd.read_csv('../input/data-science-bowl-2019/specs.csv')
ss = pd.read_csv('../input/data-science-bowl-2019/sample_submission.csv')

# train.csv / test.csv
Данные этих фалов:
- `event_id` - Уникальный идентификатор типа события. В таблице specs можно найти описание каждого такого события.
- `game_session` - Уникальный идентификатор сессии игры или видео  # same session = same game and same user and ~same time
- `timestamp` - Время возникновения события, берется со стороны клиента 
- `event_data` - Строка json формата, event_count, event_code, game_time - общие поля, наличие и содержание остальных зависит от типа события
- `installation_id` - Уникальный идентификатор приложения на устройстве - с некоторыми оговорками можно говорить, что каждый installation_id соответствует одному ребенку, пользующемуся приложением
- `event_count` - Номер события в текущей сессии, извлекается из event_data
- `event_code` - Идентификатор класса событий, объединенных общей логикой, для разных приложений(игр) может иметь разное значение, извлекается из event_data
- `game_time` - Время в миллисекундах со стратра сиссии, извлекается из event_data
- `title` - Название игры или видео
- `type` - Возможные типы: 'Game', 'Assessment', 'Activity', 'Clip'
- `world` - Показывает к какой обучающей группе (разбиение по объекту изучения) принадлежит событие. Возмежные значения: 'NONE' (Стартовый экран), 'TREETOPCITY' (Понимание длины), 'MAGMAPEAK' (Понимание вместимости, замещения), 'CRYSTALCAVES' (Понимание массы).

* Так как не все дети проходили тестирование, уберем из рассмотрения тех, на ком нельзя обучиться 

In [ ]:
print(train.shape)
train = train.loc[train['installation_id'].isin(train_labels['installation_id'])]
print(train.shape)

* Посмотрим сколько дейсвий совершали владельцы приложений

In [ ]:
train.groupby('installation_id') \
    .count()['timestamp'] \
    .plot(kind='hist',
          bins=40,
          color=my_pal[4],
          figsize=(15, 5),
         title='Count of Observations by installation_id')
plt.show()

* Похоже, что кто-то использовал приложение очень активно, лучше взглянем на логрифм от числа действий

In [ ]:
train.groupby('installation_id') \
    .count()['timestamp'] \
    .apply(np.log1p) \
    .plot(kind='hist',
          bins=50,
          color=my_pal[6],
         figsize=(15, 5),
         title='Log(Count) of Observations by installation_id')
plt.show()

* Распределение похоже на нормальное с центром в 7, т.е. большинство пользователей совершали от 400 до 3000 дейсвий в приложении

* Рассмотрим статистику для различных "обучающих групп" (поле world), чтобы выявить есть ли смысл разделять обучение сети для различных типов групп

In [ ]:
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
for world in worlds:
    train[train['world'] == world].groupby('installation_id') \
        .count()['timestamp'] \
        .apply(np.log1p) \
        .plot(kind='hist',
              bins=50,
              color=my_pal[6],
             figsize=(15, 5),
             title='Log(Count) of Observations by installation_id for {}'.format(world))
    plt.show()

* Распределение числа действий выглядит очень схоже, по этому параметру различий нет
* Рассмотрим успешность прохождения тестов для каждой группы

In [ ]:
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
for world in worlds:
    tmp = train[train['world'] == world]
    tmp = train_labels.loc[train_labels['game_session'].isin(tmp['game_session'])]
    tmp.groupby('accuracy_group').count()['game_session'].plot(figsize=(15, 5), kind='bar', title="Accuracy groups for {}".format(world))
    plt.show()

Можно наблюдать:
1. Большая часть детей решают задачу с первой попытки во всех типах заданий
2. Задачи на "вместимость" - MAGMAPEAK решаются легко, в то время как задачи на "понимание массы" - CRYSTALCAVES вызывают больше всего проблем

Получается, что имеет смысл рассматривать эти 3 типа задач отдельно.

* Рассмотрим какие коды событий являются самыми популярными в каждой категории задач

In [ ]:
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
for world in worlds:
    train[train['world'] == world].groupby('event_code') \
        .count()['timestamp'] \
        .plot(kind='bar',
              color=my_pal[6],
             figsize=(15, 5),
             title='Count of event codes for {}'.format(world))
    plt.show()


* Несмотря на то, что разные типы задач имеют различное количество и состав кодов, 5 самых популярных кодов (4070, 4030, 4020, 3110, 3010) для них одинаковы

In [ ]:
pd.options.display.max_colwidth = 200
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
for world in worlds:
    print(train[(train['event_code']==4070)&(train['world'] == world)]['event_data'].head(10))

* Код 4070, вероятно, отвечает за перемещение персонажа игры, по этим данным можно пытаться установить способ мышления ребенка, но "идеальный маршрут" уникален для каждой игры, поэтому можно сократить эти данные до количества перемещений в каждой игре

In [ ]:
pd.options.display.max_colwidth = 300
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
for world in worlds:
    print(train[(train['event_code']==4030)&(train['world'] == world)]['event_data'].head(10))

* Код 4030, скорее всего, отвечает за взаимодействие игрока с окружением, необходимым для решения задачи, из общего опять же можно выделить только количественную характеристику

In [ ]:
pd.options.display.max_colwidth = 400
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
for world in worlds:
    print(train[(train['event_code']==4020)&(train['world'] == world)]['event_data'].head(10))
    print('\n\n\n\n')

* Код 4020 для некоторых игр дает результат проверки на корректность, в целом похож на 4030

In [ ]:
pd.options.display.max_colwidth = 400
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
for world in worlds:
    print(train[(train['event_code']==3110)&(train['world'] == world)]['event_data'].head(10))
    print('\n\n\n\n')

* Коды 3110 похожи на обратную связь с пользователем, скорее всего, не несут в себе полезной информации

In [ ]:
pd.options.display.max_colwidth = 400
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
for world in worlds:
    print(train[(train['event_code']==3010)&(train['world'] == world)]['event_data'].head(10))
    print('\n\n\n\n')

* Аналогичны 3110

* Самое простое решение - придавать событиям с различными кодами различную значимость, вычислять их "функцию веса" - зависимость положительного или отрицательного вклада в результат теста от номера появления конкретного события, предположительный вид - перевернутая парабола.

# Идея обучения:
1. Создаем 3 сети - для трех типов задач
2. Для каждого ученика выделим последний сданный тест в каждом типе задач
3. На вход сети типа t поступает число различных кодов(один код для разных типов - разные входы), полученных до прохождения последнего теста типа t
4. В качестве выхода сеть выдает с какой попытки ученик пройдет тест, это сравнивается с его результатом и меняет веса
5. В качестве ответа на задачу дается взвешенная сумма трех сетей, основанная на количестве тестов каждого типа решенных учеником до предстоящего

# Шаг 1 - выделение входов и выходов


* Выделение выходов (построение аналога файла train_labels)

In [ ]:
# выделение событий - тестов
Y = train[(train['event_code'] == 4100) | (train['event_code'] == 4110)][['installation_id', 'game_session', 'timestamp', 'event_data', 'world']]
# определение был ли ответ верен
Y['event_data'] = Y['event_data'].str.contains('true')
Y['event_data'] = Y['event_data'].apply(lambda x: int(x))
Y_full = pd.DataFrame()
# выделение групп событий соответствующих одному тесту
Y_full[['installation_id', 'world', 'game_session']] = Y.groupby('game_session')[['installation_id', 'world', 'game_session']].first()
# подсчет количества попыток
Y_full['retries'] = Y.groupby('game_session')['event_data'].count()
# определение времени последней попытки
Y_full['timestamp'] = Y.groupby('game_session')['timestamp'].last()
# определение был ли тест решен
Y_full['event_data'] = Y.groupby('game_session')['event_data'].max()

Y = Y_full[['world', 'event_data', 'retries', 'game_session']]
# определение выходного класса
Y['type'] = 4 - Y['retries']
Y['type'] = Y['type'].apply(lambda x: max(x, 1))
Y['type'] = Y['event_data']*Y['type']


* Выделение частей, из которых получим входы

In [ ]:
X = pd.DataFrame(columns=['installation_id', 'event_code', 'timestamp'])
for i, world in enumerate(worlds):
    X = pd.concat([X, train[ (train['world'] == world) & (train['event_code'] != 4100) & (train['event_code'] != 4110)][['installation_id', 'event_code', 'timestamp']]])
    #разделение кодов разных типов задач
    X['event_code'] += (i+1)*10000*(2**i)

all_events = X['event_code'].unique()
X_inp = pd.DataFrame(columns=[*all_events, 'game_session', 'world'])
# переводим строки в более быстрый для сравнения формат
X['int_id'] = X['installation_id'].apply(lambda x: int(x, 16))
Y_full['int_id'] = Y_full['installation_id'].apply(lambda x: int(x, 16))
X['datetime'] = pd.to_datetime(X['timestamp'])
Y_full['datetime'] = pd.to_datetime(Y_full['timestamp'])

X_inp_lst = list('a'*Y_full.shape[0])
for i, Y_row in enumerate(Y_full.iterrows()):
    # выделяем в качестве входов для Y_row события, соответсвующие этому пользователю, совершенные ранее
    tmp = pd.DataFrame(data = [np.array(np.zeros(len(all_events)), dtype=int)], columns=all_events)
    tmp += X[(X['int_id'] == Y_row[1]['int_id'])&(X['datetime'] < Y_row[1]['datetime'])].groupby('event_code')['event_code'].count()
    X_inp_lst[i] = tmp.copy()

for i, Y_row in enumerate(Y_full.iterrows()):
    X_inp_lst[i]['game_session'] = Y_row[1]['game_session']
    X_inp_lst[i]['world'] = Y_row[1]['world']

X_inp = pd.concat(X_inp_lst)
X_inp = X_inp.fillna(0)
X_inp.to_csv("X_inp.csv")
#X_inp.head()

# Шаг 2 - выбор классификатора

* Посмотрим как себя показывают в данном задании различные методы классификации

In [ ]:
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
solvers = [SVC(), Perceptron(), SGDClassifier(), DecisionTreeClassifier(), RandomForestClassifier(n_estimators=100)]
solver_names = ['SVC', 'Perceptron', 'SGDClassifier', 'DecisionTreeClassifier', 'RandomForestClassifier']
for solver, solver_name in zip(solvers, solver_names):
    for world in worlds:
        count = X_inp[X_inp['world'] == world][all_events].shape[0]
        h = int(count*0.8)
        t = int(count*0.2)
        solver.fit(X_inp[X_inp['world'] == world][all_events].head(h), Y[Y['world'] == world][['type']].head(h))
        print(solver_name, 'Score for', world)
        print(solver.score(X_inp[X_inp['world'] == world][all_events].tail(t), Y[Y['world'] == world][['type']].tail(t)))
    print()

* Похоже, что RandomForest лучше остальных справляется с этой задачей, проверим был ли смысл в разбиении по типам тестов

In [ ]:
solvers = [SVC(), Perceptron(), SGDClassifier(), DecisionTreeClassifier(), RandomForestClassifier(n_estimators=100)]
solver_names = ['SVC', 'Perceptron', 'SGDClassifier', 'DecisionTreeClassifier', 'RandomForestClassifier']
for solver, solver_name in zip(solvers, solver_names):
    count = X_inp[all_events].shape[0]
    h = int(count*0.8)
    t = int(count*0.2)
    solver.fit(X_inp[all_events].head(h), Y[['type']].head(h))
    print(solver_name, 'Score for all')
    print(solver.score(X_inp[all_events].tail(t), Y[['type']].tail(t)))
    print()

* Почти все классификаторы показали результат хуже наименьшего по каждому типу тестов
* Проблема в том, что мы заранее не знаем какой тест будет решать пользователь, поэтому предположим, что тот же тип, в котором было совершено последнее событие

# Шаг 3 - подбор гиперпараметров

* Создаем сетку гиперпараметров

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

* Для каждого классификатора, параметры подбираются отдельно

In [ ]:
# count = X_inp[X_inp['world'] == 'CRYSTALCAVES'][all_events].shape[0]
# h = int(count*0.8)
# t = int(count*0.2)
# rf = RandomForestClassifier()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# rf_random.fit(X_inp[X_inp['world'] == 'CRYSTALCAVES'][all_events].head(h), Y[Y['world'] == 'CRYSTALCAVES'][['type']].head(h))
# print(rf_random.score(X_inp[X_inp['world'] == 'CRYSTALCAVES'][all_events].tail(t), Y[Y['world'] == 'CRYSTALCAVES'][['type']].tail(t)))
# crystal_params = rf_random.best_params_
# print(crystal_params)

In [ ]:
# count = X_inp[X_inp['world'] == 'MAGMAPEAK'][all_events].shape[0]
# h = int(count*0.8)
# t = int(count*0.2)
# rf = RandomForestClassifier()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# rf_random.fit(X_inp[X_inp['world'] == 'MAGMAPEAK'][all_events].head(h), Y[Y['world'] == 'MAGMAPEAK'][['type']].head(h))
# print(rf_random.score(X_inp[X_inp['world'] == 'MAGMAPEAK'][all_events].tail(t), Y[Y['world'] == 'MAGMAPEAK'][['type']].tail(t)))
# magma_params = rf_random.best_params_
# print(magma_params)

In [ ]:
# count = X_inp[X_inp['world'] == 'TREETOPCITY'][all_events].shape[0]
# h = int(count*0.8)
# t = int(count*0.2)
# rf = RandomForestClassifier()
# rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# rf_random.fit(X_inp[X_inp['world'] == 'TREETOPCITY'][all_events].head(h), Y[Y['world'] == 'TREETOPCITY'][['type']].head(h))
# print(rf_random.score(X_inp[X_inp['world'] == 'TREETOPCITY'][all_events].tail(t), Y[Y['world'] == 'TREETOPCITY'][['type']].tail(t)))
# treetopcity_params = rf_random.best_params_
# print(treetopcity_params)

# Шаг 3* - Подбор гиперпараметров - только результат

* Т.к. код предыдущего шага приводит к таймаутам, использую только результат

In [ ]:
crystal_params = {'n_estimators': 1000, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}
magma_params = {'n_estimators': 2000, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': False}
treetopcity_params = {'n_estimators': 1800, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': 'auto', 'max_depth': 10, 'bootstrap': False}

# Шаг 4 - обучение

In [ ]:
worlds = ['TREETOPCITY', 'MAGMAPEAK', 'CRYSTALCAVES']
classifiers = {'TREETOPCITY' : RandomForestClassifier(**treetopcity_params),
              'MAGMAPEAK' : RandomForestClassifier(**magma_params),
              'CRYSTALCAVES' : RandomForestClassifier(**crystal_params)}
for world in worlds:
    classifiers[world].fit(X_inp[X_inp['world'] == world][all_events], Y[Y['world'] == world][['type']])

# Шаг 5 - преобразование тестовых входов

In [ ]:
# преобразование id к "более бытрому" виду
test['int_id'] = test['installation_id'].apply(lambda x: int(x, 16))
all_installations = test['int_id'].unique()
test_inp_lst = list('a'*len(all_installations))
test_inp = pd.DataFrame(columns=['int_id', 'event_code', 'world'])

for i, world in enumerate(worlds):
    test_inp = pd.concat([test_inp, test[ (test['world'] == world) & (test['event_code'] != 4100) & (test['event_code'] != 4110)][['int_id', 'event_code', 'world']]])
    #разделение кодов разных типов задач
    test_inp['event_code'] += (i+1)*10000*(2**i)

# Для кажого пользователя считаем общее количество возникновения разных событий
for i, installation in enumerate(all_installations):
    tmp = pd.DataFrame(data = [np.array(np.zeros(len(all_events)), dtype=int)], columns=all_events)
    tmp += test_inp[test_inp['int_id'] == installation].groupby('event_code')['event_code'].count()
    test_inp_lst[i] = tmp.copy()

test_inp = pd.concat(test_inp_lst)
test_inp = test_inp.fillna(0)
# Определяем каким классификатором считать результат
last_world = test.groupby('int_id')['world'].last()
test_inp['world'] = last_world.values
test_inp[test_inp['world'] == 'NONE']['world'] = 'TREETOPCITY'
test_inp['int_id'] = test['installation_id'].unique()

# Шаг 6 - Предсказание

In [ ]:
for world in worlds:
    prediction = classifiers[world].predict(test_inp[test_inp['world'] == world][all_events])
    user = test_inp[test_inp['world'] == world][['int_id']]
    pd.DataFrame({'installation_id': user['int_id'], 'accuracy_group': prediction}).to_csv('submission.csv', index=False, mode='w' if world == 'TREETOPCITY' else'a')